Instructions for running notebook (on NCAR jupyterhub):  
1) Pull from github: `git clone https://github.com/josephko91/ice3d.git`
2) Make sure you are in the ice3d directory
3) Activate conda module: `module load conda`
4) Download required packages: `conda env create -f cq-conda-env.yaml -n cq`
5) Change the jupyter kernel to cq (or whatever you named it)

*NOTE: check file path for s_code.json

In [1]:
# import packages
import numpy as np
import cadquery as cq
import json
import random
import os

# (1) Load inputs

In [2]:
# Read the JSON file as a dictionary
# NOTE: when reading in from JSON, the key is read as a string
json_filepath = '/glade/u/home/joko/ice3d/s_code.json' # change directory as needed
with open(json_filepath, 'r') as json_file:
    s_code_dict = json.load(json_file)

# (2) Source Code

In [14]:
def create_bullet(a, c, hp, f_a, f_c, workplane):
    # create pyramid
    n_pyr = 6
    ri = a*np.cos(np.radians(30)) # distance between center and edge of hexagon
    theta = 90 - np.degrees(np.arctan(hp/ri))
    pyramid = workplane.polygon(n_pyr, f_a*2*a).extrude(-f_a*hp, taper=theta)
    # create cylinder 
    n_cyl = 6
    cylinder = workplane.polygon(n_cyl, f_a*2*a).extrude(f_c*2*c)
    # create bullet (union)
    bullet = cylinder.union(pyramid)
    return bullet

def calc_r0(f_r0, a, n_arms):
    '''
    linearly interpolate between perscribed limits for r0
    '''
    ymin, ymax = 0.5*a, 1*a
    xmin, xmax = 4, 12
    slope = (ymax-ymin)/(xmax-xmin)
    intercept = ymin - (slope*xmin)
    r0 = slope*(n_arms) + intercept
    r0 = f_r0 * r0 # multiply by perturbation factor
    return r0 

def calc_hp(f_hp, r0, n_arms):
    '''
    linearly interpolate: hp increases with n_arms
    '''
    ymin, ymax = 1*r0, 1.5*r0
    xmin, xmax = 4, 12
    slope = (ymax-ymin)/(xmax-xmin)
    intercept = ymin - (slope*xmin)
    hp = slope*(n_arms) + intercept
    hp = f_hp*hp # multiply by perturbation factot
    return hp

def calc_h0(f_h0, r0):
    '''
    h0 calculate as a perscribed fraction of r0
    '''
    h0 = r0/2
    h0 = f_h0*h0 # multiply by perturbation factor
    return h0

def extract_xyz(s_code):
    '''
    Convert list in format [x1, y1, z1, ..., xn, yn, zn] to separate x, y, z arrays
    '''
    x = []
    y = []
    z = []
    for i in range(0, len(s_code), 3):
        x.append(s_code[i])
        y.append(s_code[i+1])
        z.append(s_code[i+2])
    return x, y, z

# create_ros(base_params, n_arms, s_code, aspect_perturb)
def create_ros(params, n_arms, s_code, aspect_perturb):
    '''
    aspect_perturb: list in form [f_a_1,f_c_1,...,f_a_n_arms,f_c_n_arms]
    '''
    # unpack parameters
    a, c, f_r0, f_hp, f_h0 = params[0], params[1], params[2], params[3], params[4]
    r0 = calc_r0(f_r0, a, n_arms)
    hp = calc_hp(f_hp, r0, n_arms)
    h0 = calc_h0(f_h0, r0)
    # create sphere
    sphere = cq.Workplane().sphere(r0)
    # create outer shell to "place" bullets on
    # based on spherical code from Sloane et al. 
    r_outer = r0 + hp - h0
    # convert s_code list to outer_coords
    x, y, z = extract_xyz(s_code)
    outer_coords = r_outer*(np.column_stack((x, y, z)))
    # create and collect bullets in list
    bullets = []
    for i in range(len(outer_coords)):
        f_a = aspect_perturb[2*i]
        f_c = aspect_perturb[2*i + 1]
        normal_vector = tuple(outer_coords[i])
        plane = cq.Plane(origin=normal_vector, normal=normal_vector)
        workplane = cq.Workplane(plane)
        bullet = create_bullet(a, c, hp, f_a, f_c, workplane)
        bullets.append(bullet)
    # boolean union to create rosette
    ros = sphere.union(bullets[0])
    for i in range(1, n_arms):
        ros = ros.union(bullets[i])
    return ros

# (3) Main Code

## (a) Load pre-generated paramater list

In [5]:
# read test subset of params
params_path = '/glade/u/home/joko/ice3d/output/params_subset.json'
# load saved json
with open(params_path, 'rb') as file:
    params_subset = json.load(file)

## (b) Generate random rosette from parameter list

In [21]:
# Specify parameters
params = random.choice(params_subset)
base_params = params[0][:5]
n_arms = params[0][5]
aspect_perturb = params[1]
s_code = params[2]
print(f'base params: {base_params}')
print(f'n_arms: {n_arms}')
print(f'aspect_perturb: {aspect_perturb}')
print(f's_code: {s_code}')
# create rosette
ros = create_ros(base_params, n_arms, s_code, aspect_perturb)
print(f'rosette surface area: {ros.val().Area()}')
print(f'rosette volume: {ros.val().Volume()}')
ros

base params: [18.079919627168973, 111.4796925553237, 0.9138131147559592, 1.1285629769805177, 0.9584662764341794]
n_arms: 8
aspect_perturb: [0.8820613228059757, 1.1718178230243317, 1.111322103508085, 1.0738295637005848, 0.9249222608723375, 0.8476980209476982, 1.1253500232591176, 1.0950827364581162, 1.0870231358068252, 1.1531155410890557, 0.8713674688377926, 1.1369541148857956, 1.1941188370630758, 0.837081738933721, 1.0385683661344183, 0.9921272992776091]
s_code: [-0.7144459713919096, 0.626995496618152, 0.3105537009639836, 0.14359586600391344, -0.8046215847931131, -0.576163459924009, 0.7139971635382028, 0.6251919536226297, -0.31518735949419113, -0.051598708757863314, -0.8824354947322057, 0.46759509288619394, -0.8300366346251121, -0.401170235122092, -0.38742951311471285, 0.6802823986649743, 0.5913358555313404, 0.4330563058421463, 0.850704591853586, -0.4538267172604457, 0.265222563330933, -0.40992841232964033, 0.8012487749979252, -0.43584297324750393]
rosette surface area: 220206.198928508

## (c) Save rosette as STL (mesh format)

In [ ]:
# if you want to save file
save_dir = '/glade/u/home/joko/ice3d/output'
save_filename = 'ros-test.stl'
save_filepath = os.path.join(save_dir, save_filename)
cq.exporters.export(ros, save_filepath)